The work is inspired by [Memo on Fresnel equations](https://seblagarde.wordpress.com/2013/04/29/memo-on-fresnel-equations/), check it out if you don't.

Snell's law dielectric-dielectric interface :
$$\theta_i = \theta_o$$
$$\eta_i \sin \theta_i = \eta_t \sin \theta_t$$

Snell's law dielectric-conductorinterface :
$$\theta_i = \theta_o$$
$$\eta_i \sin \theta_i = (\eta_t + i\kappa_t) \sin \theta_t$$

Fresnel equation

\begin{equation}
\bar\eta_i = \eta_i + i\kappa_i \\
\bar\eta_t = \eta_t + i\kappa_t
\end{equation}

\begin{align}
& r_\perp = \frac {a^2 + b^2 - 2a\cos\theta + \cos{^2}\theta} 
                  {a^2 + b^2 + 2a\cos\theta + \cos{^2}\theta} \\
& r_\parallel = {r_\perp} \frac {\cos{^2}\theta \space {(a^2 + b^2)} - {2a} \space {\cos\theta} \space {\sin{^2}\theta + \sin{^4}\theta}}
                                {\cos{^2}\theta \space {(a^2 + b^2)} + {2a} \space {\cos\theta} \space {\sin{^2}\theta + \sin{^4}\theta}}
\end{align}